**Anna Smith**

**GitHub username: acs-233**

**Imperial College London - MSc EDSML - IRP**
# **XGBoost model to predict $\textnormal{BECO}_2\textnormal{N}$ data**

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score

In [2]:
from bayareaco2.models.data import fit_scaler, scale_features

## Reading in Data

In [3]:
path = Path.cwd().parent.parent.parent / 'Data' / 'CO2' / 'balanced_daily_avg_BEACO2N.csv'
co2 = pd.read_csv(path)

path = Path.cwd().parent.parent.parent / 'Data' / 'features.csv'
features = pd.read_csv(path)

# Merging data
df = co2.merge(features, on='node_id')

In [4]:
# Confirming no missing values
np.max(df.isna().sum())

0

In [5]:
df

,Unnamed: 0,node_id,date,timestamp,co2,temp,pressure,rh,year,month,...,avg_pop_dens_50m,avg_pop_dens_100m,avg_pop_dens_200m,avg_pop_dens_300m,avg_pop_dens_500m,avg_pop_dens_1000m,avg_pop_dens_2000m,avg_pop_dens_3000m,avg_pop_dens_4000m,avg_pop_dens_5000m
0,1539,4,2022-01-01,2022-01-01 11:30:00.000000000,444.400204,7.547035,999.754739,60.628520,2022,1,...,5208.800000,5208.800000,5179.744961,4934.744803,4677.358739,4494.435919,4269.319433,4436.773639,5443.439375,6707.038196
1,1540,4,2022-01-02,2022-01-02 11:30:00.000000000,453.541155,8.937224,1001.820275,63.048644,2022,1,...,5208.800000,5208.800000,5179.744961,4934.744803,4677.358739,4494.435919,4269.319433,4436.773639,5443.439375,6707.038196
2,1541,4,2022-01-03,2022-01-03 11:30:00.000000000,441.211655,10.741178,1000.370385,76.591252,2022,1,...,5208.800000,5208.800000,5179.744961,4934.744803,4677.358739,4494.435919,4269.319433,4436.773639,5443.439375,6707.038196
3,1542,4,2022-01-04,2022-01-04 11:30:00.000000000,433.777005,13.741635,1003.960120,80.556825,2022,1,...,5208.800000,5208.800000,5179.744961,4934.744803,4677.358739,4494.435919,4269.319433,4436.773639,5443.439375,6707.038196
4,1543,4,2022-01-05,2022-01-05 11:30:00.000000000,436.049086,14.247229,1002.731049,77.232257,2022,1,...,5208.800000,5208.800000,5179.744961,4934.744803,4677.358739,4494.435919,4269.319433,4436.773639,5443.439375,6707.038196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17384,43738,286,2023-10-08,2023-10-08 11:30:00.000000000,423.624269,24.253212,1010.376829,46.244989,2023,10,...,15127.053329,15953.607262,16300.528823,16150.012015,16387.774742,14981.046811,12583.537953,12283.743192,10896.300370,9445.255071
17385,43739,286,2023-10-09,2023-10-09 11:30:00.000000000,416.561727,24.336228,1009.992955,46.518173,2023,10,...,15127.053329,15953.607262,16300.528823,16150.012015,16387.774742,14981.046811,12583.537953,12283.743192,10896.300370,9445.255071
17386,43740,286,2023-10-10,2023-10-10 11:15:39.130434816,415.588174,24.621320,1008.725354,48.531213,2023,10,...,15127.053329,15953.607262,16300.528823,16150.012015,16387.774742,14981.046811,12583.537953,12283.743192,10896.300370,9445.255071
17387,43741,286,2023-10-11,2023-10-11 11:30:00.000000000,420.917376,24.556401,1009.965181,38.998069,2023,10,...,15127.053329,15953.607262,16300.528823,16150.012015,16387.774742,14981.046811,12583.537953,12283.743192,10896.300370,9445.255071


In [6]:
# Defining dependent variable
y = df['co2'].copy()

In [7]:
# Combining feature labels
feature_vars = features.drop(columns='node_id').columns.to_list()
feature_vars.append('temp')
feature_vars.append('pressure')
feature_vars.append('rh')

In [8]:
node_id = df['node_id'].copy()

In [9]:
# Defining dependent variables
X = df[feature_vars].copy()

In [10]:
# Inspecting X -- 123 features
X

,Built_Area_area_50m,Rangeland_area_50m,Trees_area_50m,Water_area_50m,Bare_Ground_area_50m,Crops_area_50m,Flooded_Vegetation_area_50m,Built_Area_area_100m,Rangeland_area_100m,Trees_area_100m,...,avg_pop_dens_300m,avg_pop_dens_500m,avg_pop_dens_1000m,avg_pop_dens_2000m,avg_pop_dens_3000m,avg_pop_dens_4000m,avg_pop_dens_5000m,temp,pressure,rh
0,7841.371226,0.0,0.0,0.0,0,0,0,31365.484905,0.0,0.0,...,4934.744803,4677.358739,4494.435919,4269.319433,4436.773639,5443.439375,6707.038196,7.547035,999.754739,60.628520
1,7841.371226,0.0,0.0,0.0,0,0,0,31365.484905,0.0,0.0,...,4934.744803,4677.358739,4494.435919,4269.319433,4436.773639,5443.439375,6707.038196,8.937224,1001.820275,63.048644
2,7841.371226,0.0,0.0,0.0,0,0,0,31365.484905,0.0,0.0,...,4934.744803,4677.358739,4494.435919,4269.319433,4436.773639,5443.439375,6707.038196,10.741178,1000.370385,76.591252
3,7841.371226,0.0,0.0,0.0,0,0,0,31365.484905,0.0,0.0,...,4934.744803,4677.358739,4494.435919,4269.319433,4436.773639,5443.439375,6707.038196,13.741635,1003.960120,80.556825
4,7841.371226,0.0,0.0,0.0,0,0,0,31365.484905,0.0,0.0,...,4934.744803,4677.358739,4494.435919,4269.319433,4436.773639,5443.439375,6707.038196,14.247229,1002.731049,77.232257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17384,7841.371226,0.0,0.0,0.0,0,0,0,31365.484906,0.0,0.0,...,16150.012015,16387.774742,14981.046811,12583.537953,12283.743192,10896.300370,9445.255071,24.253212,1010.376829,46.244989
17385,7841.371226,0.0,0.0,0.0,0,0,0,31365.484906,0.0,0.0,...,16150.012015,16387.774742,14981.046811,12583.537953,12283.743192,10896.300370,9445.255071,24.336228,1009.992955,46.518173
17386,7841.371226,0.0,0.0,0.0,0,0,0,31365.484906,0.0,0.0,...,16150.012015,16387.774742,14981.046811,12583.537953,12283.743192,10896.300370,9445.255071,24.621320,1008.725354,48.531213
17387,7841.371226,0.0,0.0,0.0,0,0,0,31365.484906,0.0,0.0,...,16150.012015,16387.774742,14981.046811,12583.537953,12283.743192,10896.300370,9445.255071,24.556401,1009.965181,38.998069


In [11]:
# Inspecting y
y

0        444.400204
1        453.541155
2        441.211655
3        433.777005
4        436.049086
            ...    
17384    423.624269
17385    416.561727
17386    415.588174
17387    420.917376
17388    434.061157
Name: co2, Length: 17389, dtype: float64

In [12]:
node_id

0          4
1          4
2          4
3          4
4          4
        ... 
17384    286
17385    286
17386    286
17387    286
17388    286
Name: node_id, Length: 17389, dtype: int64

## Preparing Data

In [13]:
# Checking for zero columns
zero_columns = [col for col in X.columns if (X[col] == 0).all()]
print(f"{len(zero_columns)} columns with all zeros:")
zero_columns

18 columns with all zeros:


['Rangeland_area_50m',
 'Bare_Ground_area_50m',
 'Crops_area_50m',
 'Flooded_Vegetation_area_50m',
 'Bare_Ground_area_100m',
 'Crops_area_100m',
 'Flooded_Vegetation_area_100m',
 'Bare_Ground_area_200m',
 'Crops_area_200m',
 'Flooded_Vegetation_area_200m',
 'Bare_Ground_area_300m',
 'Crops_area_300m',
 'Flooded_Vegetation_area_300m',
 'Bare_Ground_area_500m',
 'total_AADT_50m',
 'total_AADT_100m',
 'total_AADT_200m',
 'total_road_length_50m']

In [14]:
# Dropping zero columns
X = X.drop(columns = zero_columns)

In [15]:
# Standardizing the data
scaler = fit_scaler(X)
X_scaled = scale_features(scaler, X)

In [16]:
# Splitting data
X_train_full, X_test_full, y_train_full, y_test_full, node_id_train_full, node_id_test_full = train_test_split(X_scaled, y, node_id, test_size=0.2, random_state=42)

In [17]:
X_train, X_val, y_train, y_val, node_id_train, node_id_val= train_test_split(X_train_full, y_train_full, node_id_train_full, test_size=0.2, random_state=42)

In [18]:
print(f"Full training set length: {len(X_train_full)}")
print(f"Full training set length: {len(X_test_full)}")
print(f"Training set length: {len(X_train)}")
print(f"Validation set length: {len(X_val)}")

Full training set length: 13911
Full training set length: 3478
Training set length: 11128
Validation set length: 2783


## Training

In [21]:
# # Define the XGBoost model
# xgboost_model = xgb.XGBRegressor(objective='reg:squarederror')

# # Define the parameter grid for optimization
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [5, 10, 15],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'subsample': [0.8],
#     'colsample_bytree': [0.8]
# }

# # Perform Grid Search with 10-fold cross-validation
# grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
# grid_search.fit(X_train, y_train)

# # Get the best parameters
# best_params = grid_search.best_params_
# print(f"Best parameters: {best_params}")

In [22]:
best_params = {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}

In [23]:
# Train the model with the best parameters
best_xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
best_xgboost_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [24]:
y_pred = best_xgboost_model.predict(X_val)

# Calculate evaluation metrics
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
n = X_val.shape[0]
p = X_val.shape[1]

# Print evaluation metrics
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R²: {r2:.2f}")

Mean Squared Error (MSE): 145.11
Root Mean Squared Error (RMSE): 12.05
Mean Absolute Error (MAE): 8.79
R²: 0.60


In [25]:
# 10-fold cross-validation for MSE
cv_scores = cross_val_score(best_xgboost_model, X_train_full, y_train_full, cv=10, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()
cv_rmse = np.sqrt(cv_mse)
print(f"10-fold CV Mean Squared Error (MSE): {cv_mse:.2f}")
print(f"10-fold CV Root Mean Squared Error (RMSE): {cv_rmse:.2f}")

# 10-fold cross-validation for MAE
cv_mae_scores = cross_val_score(best_xgboost_model, X_train_full, y_train_full, cv=10, scoring='neg_mean_absolute_error')
cv_mae = -cv_mae_scores.mean()
print(f"10-fold CV Mean Absolute Error (MAE): {cv_mae:.2f}")

# 10-fold cross-validation for R²
cv_r2_scores = cross_val_score(best_xgboost_model, X_train_full, y_train_full, cv=10, scoring='r2')
cv_r2 = cv_r2_scores.mean()
print(f"10-fold CV R²: {cv_r2:.2f}")

10-fold CV Mean Squared Error (MSE): 157.69
10-fold CV Root Mean Squared Error (RMSE): 12.56
10-fold CV Mean Absolute Error (MAE): 9.01
10-fold CV R²: 0.58


## Training on Full Training Set

In [26]:
# Train the model with the best parameters
full_xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
full_xgboost_model.fit(X_train_full, y_train_full)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [27]:
y_pred_test = full_xgboost_model.predict(X_test_full)

# Calculate evaluation metrics
mse = mean_squared_error(y_test_full, y_pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_full, y_pred_test)
r2 = r2_score(y_test_full, y_pred_test)
n = X_val.shape[0]
p = X_val.shape[1]

# Print evaluation metrics
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R²: {r2:.2f}")

Mean Squared Error (MSE): 155.81
Root Mean Squared Error (RMSE): 12.48
Mean Absolute Error (MAE): 9.06
R²: 0.59


In [28]:
# 10-fold cross-validation for MSE
cv_scores = cross_val_score(full_xgboost_model, X_train_full, y_train_full, cv=10, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()
cv_rmse = np.sqrt(cv_mse)
print(f"10-fold CV Mean Squared Error (MSE): {cv_mse:.2f}")
print(f"10-fold CV Root Mean Squared Error (RMSE): {cv_rmse:.2f}")

# 10-fold cross-validation for MAE
cv_mae_scores = cross_val_score(full_xgboost_model, X_train_full, y_train_full, cv=10, scoring='neg_mean_absolute_error')
cv_mae = -cv_mae_scores.mean()
print(f"10-fold CV Mean Absolute Error (MAE): {cv_mae:.2f}")

# 10-fold cross-validation for R²
cv_r2_scores = cross_val_score(full_xgboost_model, X_train_full, y_train_full, cv=10, scoring='r2')
cv_r2 = cv_r2_scores.mean()
print(f"10-fold CV R²: {cv_r2:.2f}")

10-fold CV Mean Squared Error (MSE): 157.69
10-fold CV Root Mean Squared Error (RMSE): 12.56
10-fold CV Mean Absolute Error (MAE): 9.01
10-fold CV R²: 0.58
